d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px; height: 163px">
</div>

-sandbox
<img src="https://files.training.databricks.com/images/Apache-Spark-Logo_TM_200px.png" style="float: left: margin: 20px"/>

# Table Management

Apache Spark&trade; and Databricks&reg; allow you to access and optimize data in managed and unmanaged tables.

## In this lesson you:
* Write to managed and unmanaged tables
* Explore the effect of dropping tables on the metadata and underlying data

## Audience
* Primary Audience: Data Engineers
* Additional Audiences: Data Scientists and Data Pipeline Engineers

## Prerequisites
* Web browser: Chrome
* A cluster configured with **8 cores** and **DBR 6.2**
* Course: ETL Part 1 from <a href="https://academy.databricks.com/" target="_blank">Databricks Academy</a>

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Classroom-Setup

For each lesson to execute correctly, please make sure to run the **`Classroom-Setup`** cell at the<br/>
start of each lesson (see the next cell) and the **`Classroom-Cleanup`** cell at the end of each lesson.

In [4]:
%run "./Includes/Classroom-Setup"

<iframe  
src="//fast.wistia.net/embed/iframe/gyw20pwx6j?videoFoam=true"
style="border:1px solid #1cb1c2;"
allowtransparency="true" scrolling="no" class="wistia_embed"
name="wistia_embed" allowfullscreen mozallowfullscreen webkitallowfullscreen
oallowfullscreen msallowfullscreen width="640" height="360" ></iframe>
<div>
<a target="_blank" href="https://fast.wistia.net/embed/iframe/gyw20pwx6j?seo=false">
  <img alt="Opens in new tab" src="https://files.training.databricks.com/static/images/external-link-icon-16x16.png"/>&nbsp;Watch full-screen.</a>
</div>

-sandbox
### Optimization of Data Storage with Managed and Unmanaged Tables

A **managed table** is a table that manages both the data itself as well as the metadata.  In this case, a `DROP TABLE` command removes both the metadata for the table as well as the data itself.  

**Unmanaged tables** manage the metadata from a table such as the schema and data location, but the data itself sits in a different location, often backed by a blob store like the Azure Blob or S3. Dropping an unmanaged table drops only the metadata associated with the table while the data itself remains in place.

<div><img src="https://files.training.databricks.com/images/eLearning/ETL-Part-2/managed-and-unmanaged-tables.png" style="height: 400px; margin: 20px"/></div>

### Writing to a Managed Table

Managed tables allow access to data using the Spark SQL API.

Create a DataFrame.

In [9]:
df = spark.range(1, 100)

display(df)

id
1
2
3
4
5
6
7
8
9
10


Register the table.

In [11]:
df.write.mode("OVERWRITE").saveAsTable("myTableManaged")

Use `DESCRIBE EXTENDED` to describe the contents of the table.  Scroll down to see the table `Type`.

Notice the location is also `dbfs:/user/hive/warehouse/< your database >/mytablemanaged`.

In [13]:
%sql
DESCRIBE EXTENDED myTableManaged

col_name,data_type,comment
id,bigint,null
,,
# Detailed Table Information,,
Database,tbresee_umich_edu_etl_part_2_etl2_07_table_management_psp,
Table,mytablemanaged,
Owner,root,
Created Time,Fri Apr 17 22:35:26 UTC 2020,
Last Access,Thu Jan 01 00:00:00 UTC 1970,
Created By,Spark 2.4.5,
Type,MANAGED,


### Writing to an Unmanaged Table

Write to an unmanaged table by adding an `.option()` that includes a path.

In [15]:
unmanagedPath = f"{workingDir}/myTableUnmanaged"

df.write.mode("OVERWRITE").option('path', unmanagedPath).saveAsTable("myTableUnmanaged")

-sandbox
Now examine the table type and location of the data.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> An external table is the same as an unmanaged table.

In [17]:
%sql
DESCRIBE EXTENDED myTableUnmanaged

col_name,data_type,comment
id,bigint,null
,,
# Detailed Table Information,,
Database,tbresee_umich_edu_etl_part_2_etl2_07_table_management_psp,
Table,mytableunmanaged,
Owner,root,
Created Time,Fri Apr 17 22:35:28 UTC 2020,
Last Access,Thu Jan 01 00:00:00 UTC 1970,
Created By,Spark 2.4.5,
Type,EXTERNAL,


### Dropping Tables

Take a look at how dropping tables operates differently in the two cases below.

Look at the files backing up the managed table.

In [20]:
hivePath = f"dbfs:/user/hive/warehouse/{databaseName}.db/mytablemanaged"

display(dbutils.fs.ls(hivePath))

path,name,size
dbfs:/user/hive/warehouse/tbresee_umich_edu_etl_part_2_etl2_07_table_management_psp.db/mytablemanaged/_SUCCESS,_SUCCESS,0
dbfs:/user/hive/warehouse/tbresee_umich_edu_etl_part_2_etl2_07_table_management_psp.db/mytablemanaged/_committed_6369770338014875865,_committed_6369770338014875865,840
dbfs:/user/hive/warehouse/tbresee_umich_edu_etl_part_2_etl2_07_table_management_psp.db/mytablemanaged/_started_6369770338014875865,_started_6369770338014875865,0
dbfs:/user/hive/warehouse/tbresee_umich_edu_etl_part_2_etl2_07_table_management_psp.db/mytablemanaged/part-00000-tid-6369770338014875865-8ba39550-5deb-4758-9314-fac663c6c70e-18211-1-c000.snappy.parquet,part-00000-tid-6369770338014875865-8ba39550-5deb-4758-9314-fac663c6c70e-18211-1-c000.snappy.parquet,494
dbfs:/user/hive/warehouse/tbresee_umich_edu_etl_part_2_etl2_07_table_management_psp.db/mytablemanaged/part-00001-tid-6369770338014875865-8ba39550-5deb-4758-9314-fac663c6c70e-18212-1-c000.snappy.parquet,part-00001-tid-6369770338014875865-8ba39550-5deb-4758-9314-fac663c6c70e-18212-1-c000.snappy.parquet,494
dbfs:/user/hive/warehouse/tbresee_umich_edu_etl_part_2_etl2_07_table_management_psp.db/mytablemanaged/part-00002-tid-6369770338014875865-8ba39550-5deb-4758-9314-fac663c6c70e-18213-1-c000.snappy.parquet,part-00002-tid-6369770338014875865-8ba39550-5deb-4758-9314-fac663c6c70e-18213-1-c000.snappy.parquet,499
dbfs:/user/hive/warehouse/tbresee_umich_edu_etl_part_2_etl2_07_table_management_psp.db/mytablemanaged/part-00003-tid-6369770338014875865-8ba39550-5deb-4758-9314-fac663c6c70e-18214-1-c000.snappy.parquet,part-00003-tid-6369770338014875865-8ba39550-5deb-4758-9314-fac663c6c70e-18214-1-c000.snappy.parquet,494
dbfs:/user/hive/warehouse/tbresee_umich_edu_etl_part_2_etl2_07_table_management_psp.db/mytablemanaged/part-00004-tid-6369770338014875865-8ba39550-5deb-4758-9314-fac663c6c70e-18215-1-c000.snappy.parquet,part-00004-tid-6369770338014875865-8ba39550-5deb-4758-9314-fac663c6c70e-18215-1-c000.snappy.parquet,493
dbfs:/user/hive/warehouse/tbresee_umich_edu_etl_part_2_etl2_07_table_management_psp.db/mytablemanaged/part-00005-tid-6369770338014875865-8ba39550-5deb-4758-9314-fac663c6c70e-18216-1-c000.snappy.parquet,part-00005-tid-6369770338014875865-8ba39550-5deb-4758-9314-fac663c6c70e-18216-1-c000.snappy.parquet,499
dbfs:/user/hive/warehouse/tbresee_umich_edu_etl_part_2_etl2_07_table_management_psp.db/mytablemanaged/part-00006-tid-6369770338014875865-8ba39550-5deb-4758-9314-fac663c6c70e-18217-1-c000.snappy.parquet,part-00006-tid-6369770338014875865-8ba39550-5deb-4758-9314-fac663c6c70e-18217-1-c000.snappy.parquet,494


Drop the table.

In [22]:
%sql
DROP TABLE myTableManaged

-sandbox
Next look at the underlying data.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> This command will throw an error.

In [24]:
try:
  display(dbutils.fs.ls(hivePath))
  
except Exception as e:
  print(e)

An error occurred while calling z:com.databricks.backend.daemon.dbutils.FSUtils.ls.
: java.io.FileNotFoundException: /user/hive/warehouse/tbresee_umich_edu_etl_part_2_etl2_07_table_management_psp.db/mytablemanaged
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:115)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:63)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV1.listStatus(DatabricksFileSystemV1.scala:177)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystem.listStatus(DatabricksFileSystem.scala:150)
	at com.databricks.backend.daemon.dbutils.FSUtils$$anonfun$ls$1.apply(DBUtilsCore.scala:86)
	at com.databricks.backend.daemon.dbutils.FSUtils$$anonfun$ls$1.apply(DBUtilsCore.scala:85)
	at com.databricks.backend.daemon.dbutils.FSUtils$.com$databricks$backend$daemon$dbutils$FSUtils$$withFsSafetyCheck(DBUtilsCore.scala:81)
	at com.databricks.backend.daemon.dbutils.FSUtils$.ls(DBUtilsCore.scala:85)
	at com.databricks.backend.daemon.dbutils.FSUtils.ls(DBUtilsCore.scala)
	at sun.reflect.GeneratedMethodAccessor699.invoke(Unknown Source)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)

The data was deleted so spark will not find the underlying data. Perform the same operation with the unmanaged table.

In [26]:
display(dbutils.fs.ls(unmanagedPath))

path,name,size
dbfs:/user/tbresee@umich.edu/etl_part_2/etl2_07_table_management_psp/myTableUnmanaged/_SUCCESS,_SUCCESS,0
dbfs:/user/tbresee@umich.edu/etl_part_2/etl2_07_table_management_psp/myTableUnmanaged/_committed_5569118828152684927,_committed_5569118828152684927,840
dbfs:/user/tbresee@umich.edu/etl_part_2/etl2_07_table_management_psp/myTableUnmanaged/_started_5569118828152684927,_started_5569118828152684927,0
dbfs:/user/tbresee@umich.edu/etl_part_2/etl2_07_table_management_psp/myTableUnmanaged/part-00000-tid-5569118828152684927-eeb1d035-5652-40d8-9765-951239559e95-18219-1-c000.snappy.parquet,part-00000-tid-5569118828152684927-eeb1d035-5652-40d8-9765-951239559e95-18219-1-c000.snappy.parquet,494
dbfs:/user/tbresee@umich.edu/etl_part_2/etl2_07_table_management_psp/myTableUnmanaged/part-00001-tid-5569118828152684927-eeb1d035-5652-40d8-9765-951239559e95-18220-1-c000.snappy.parquet,part-00001-tid-5569118828152684927-eeb1d035-5652-40d8-9765-951239559e95-18220-1-c000.snappy.parquet,494
dbfs:/user/tbresee@umich.edu/etl_part_2/etl2_07_table_management_psp/myTableUnmanaged/part-00002-tid-5569118828152684927-eeb1d035-5652-40d8-9765-951239559e95-18221-1-c000.snappy.parquet,part-00002-tid-5569118828152684927-eeb1d035-5652-40d8-9765-951239559e95-18221-1-c000.snappy.parquet,499
dbfs:/user/tbresee@umich.edu/etl_part_2/etl2_07_table_management_psp/myTableUnmanaged/part-00003-tid-5569118828152684927-eeb1d035-5652-40d8-9765-951239559e95-18222-1-c000.snappy.parquet,part-00003-tid-5569118828152684927-eeb1d035-5652-40d8-9765-951239559e95-18222-1-c000.snappy.parquet,494
dbfs:/user/tbresee@umich.edu/etl_part_2/etl2_07_table_management_psp/myTableUnmanaged/part-00004-tid-5569118828152684927-eeb1d035-5652-40d8-9765-951239559e95-18223-1-c000.snappy.parquet,part-00004-tid-5569118828152684927-eeb1d035-5652-40d8-9765-951239559e95-18223-1-c000.snappy.parquet,493
dbfs:/user/tbresee@umich.edu/etl_part_2/etl2_07_table_management_psp/myTableUnmanaged/part-00005-tid-5569118828152684927-eeb1d035-5652-40d8-9765-951239559e95-18224-1-c000.snappy.parquet,part-00005-tid-5569118828152684927-eeb1d035-5652-40d8-9765-951239559e95-18224-1-c000.snappy.parquet,499
dbfs:/user/tbresee@umich.edu/etl_part_2/etl2_07_table_management_psp/myTableUnmanaged/part-00006-tid-5569118828152684927-eeb1d035-5652-40d8-9765-951239559e95-18225-1-c000.snappy.parquet,part-00006-tid-5569118828152684927-eeb1d035-5652-40d8-9765-951239559e95-18225-1-c000.snappy.parquet,494


Drop the unmanaged table.

In [28]:
%sql
DROP TABLE myTableUnmanaged

See if the data is still there.

In [30]:
display(dbutils.fs.ls(unmanagedPath))

path,name,size
dbfs:/user/tbresee@umich.edu/etl_part_2/etl2_07_table_management_psp/myTableUnmanaged/_SUCCESS,_SUCCESS,0
dbfs:/user/tbresee@umich.edu/etl_part_2/etl2_07_table_management_psp/myTableUnmanaged/_committed_5569118828152684927,_committed_5569118828152684927,840
dbfs:/user/tbresee@umich.edu/etl_part_2/etl2_07_table_management_psp/myTableUnmanaged/_started_5569118828152684927,_started_5569118828152684927,0
dbfs:/user/tbresee@umich.edu/etl_part_2/etl2_07_table_management_psp/myTableUnmanaged/part-00000-tid-5569118828152684927-eeb1d035-5652-40d8-9765-951239559e95-18219-1-c000.snappy.parquet,part-00000-tid-5569118828152684927-eeb1d035-5652-40d8-9765-951239559e95-18219-1-c000.snappy.parquet,494
dbfs:/user/tbresee@umich.edu/etl_part_2/etl2_07_table_management_psp/myTableUnmanaged/part-00001-tid-5569118828152684927-eeb1d035-5652-40d8-9765-951239559e95-18220-1-c000.snappy.parquet,part-00001-tid-5569118828152684927-eeb1d035-5652-40d8-9765-951239559e95-18220-1-c000.snappy.parquet,494
dbfs:/user/tbresee@umich.edu/etl_part_2/etl2_07_table_management_psp/myTableUnmanaged/part-00002-tid-5569118828152684927-eeb1d035-5652-40d8-9765-951239559e95-18221-1-c000.snappy.parquet,part-00002-tid-5569118828152684927-eeb1d035-5652-40d8-9765-951239559e95-18221-1-c000.snappy.parquet,499
dbfs:/user/tbresee@umich.edu/etl_part_2/etl2_07_table_management_psp/myTableUnmanaged/part-00003-tid-5569118828152684927-eeb1d035-5652-40d8-9765-951239559e95-18222-1-c000.snappy.parquet,part-00003-tid-5569118828152684927-eeb1d035-5652-40d8-9765-951239559e95-18222-1-c000.snappy.parquet,494
dbfs:/user/tbresee@umich.edu/etl_part_2/etl2_07_table_management_psp/myTableUnmanaged/part-00004-tid-5569118828152684927-eeb1d035-5652-40d8-9765-951239559e95-18223-1-c000.snappy.parquet,part-00004-tid-5569118828152684927-eeb1d035-5652-40d8-9765-951239559e95-18223-1-c000.snappy.parquet,493
dbfs:/user/tbresee@umich.edu/etl_part_2/etl2_07_table_management_psp/myTableUnmanaged/part-00005-tid-5569118828152684927-eeb1d035-5652-40d8-9765-951239559e95-18224-1-c000.snappy.parquet,part-00005-tid-5569118828152684927-eeb1d035-5652-40d8-9765-951239559e95-18224-1-c000.snappy.parquet,499
dbfs:/user/tbresee@umich.edu/etl_part_2/etl2_07_table_management_psp/myTableUnmanaged/part-00006-tid-5569118828152684927-eeb1d035-5652-40d8-9765-951239559e95-18225-1-c000.snappy.parquet,part-00006-tid-5569118828152684927-eeb1d035-5652-40d8-9765-951239559e95-18225-1-c000.snappy.parquet,494


## Review
**Question:** What happens to the original data when I delete a managed table?  What about an unmanaged table?  
**Answer:** Deleting a managed table deletes both the metadata and the data itself. Deleting an unmanaged table does not delete the original data.

**Question:** What is a metastore?  
**Answer:** A metastore is a repository of metadata such as the location of where data is and the schema information. A metastore does not include the data itself.

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Classroom-Cleanup<br>

Run the **`Classroom-Cleanup`** cell below to remove any artifacts created by this lesson.

In [33]:
%run "./Includes/Classroom-Cleanup"

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> All done!</h2>

Thank you for your participation!

## Additional Topics & Resources

**Q:** Where can I find out more about connecting to my own metastore?  
**A:** Take a look at the <a href="https://docs.databricks.com/user-guide/advanced/external-hive-metastore.html" target="_blank">Databricks documentation for more details</a>

**Q:** Where can I find out more about Spark Tables?  
**A:** Take a look at the <a href="https://docs.databricks.com/user-guide/tables.html" target="_blank">Databricks documentation for more details</a>

-sandbox
&copy; 2020 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>